In [1]:
from train_dnn import get_data
from data import adult, compas, bank, german, loans_default, census
from models.trainer import STDTrainer, SenSeiTrainer
from models.model import MLP
from seeker.random import RandomSelectPairSeeker, RandomSelectSeeker, RangeGenSeeker, DistributionGenSeeker
from seeker.gradiant_based import WhiteboxSeeker, BlackboxSeeker, FoolSeeker
from distances.normalized_mahalanobis_distances import ProtectedSEDistances
from distances.sensitive_subspace_distances import LogisticRegSensitiveSubspace
from distances.binary_distances import BinaryDistance
from inFairness.distances import SquaredEuclideanDistance
from utils import UnfairMetric, load_model
import torch
import random

torch.set_default_dtype(torch.float64)

%load_ext autoreload
%autoreload 2

In [2]:
load = True
norm = False

data_name = 'census'
use_sensitive_attr = True
trainer_name = 'std'
# trainer_name = 'sensei'
rho=0.1
note = '_rho={rho}' if trainer_name == 'sensei' else ''
device = 'cuda:1'
sensitive_vars = ['sex_Male']

data_choices = {
    'adult': adult,
    'german': german,
    'loans_default': loans_default,
    'census': census
}
data = data_choices[data_name]

dataset, train_dl, test_dl = get_data(data, 0, sensitive_vars)
dataset.use_sensitive_attr = use_sensitive_attr
feature_dim = dataset.dim_feature()
output_dim = 2

data_gen = data.Generator(use_sensitive_attr, sensitive_vars, device)

In [3]:
# fit distances for LRTrainer
if trainer_name == 'sensei':
    train_distance_x = LogisticRegSensitiveSubspace()
    train_distance_y = SquaredEuclideanDistance()

    dataset.use_sensitive_attr = True
    all_X_train = []
    for x, _ in train_dl:
        all_X_train.append(x)
    all_X_train = torch.concat(all_X_train, dim=0)
    dataset.use_sensitive_attr = use_sensitive_attr

    if use_sensitive_attr:
        train_distance_x.fit(all_X_train, data_gen=data_gen, sensitive_idxs=dataset.sensitive_idxs)
    else:
        sensitive_ = all_X_train[:, dataset.sensitive_idxs]
        no_sensitive = all_X_train[:, [i for i in range(all_X_train.shape[1]) if i not in dataset.sensitive_idxs]]
        train_distance_x.fit(no_sensitive, data_gen=data_gen, data_SensitiveAttrs=sensitive_)
    train_distance_y.fit(output_dim)
    train_distance_x.to(device)
    train_distance_y.to(device)

In [4]:
model = MLP(input_size=feature_dim, output_size=output_dim, data_gen=data_gen, n_layers=4, norm=norm)

if load:
    load_model(model, data_name, trainer_name, use_sensitive_attr=use_sensitive_attr, \
            sensitive_vars=sensitive_vars, id=0, note=note)
    model = model.to(device)
else:
    if trainer_name == 'std':
        trainer = STDTrainer(model, train_dl, test_dl, device=device, epochs=100, lr=1e-4)
    elif trainer_name == 'sensei':
        trainer = SenSeiTrainer(model, train_dl, test_dl, device=device, epochs=1000, lr=1e-3, distance_x=train_distance_x, distance_y=train_distance_y, rho=rho)
    trainer.train()

In [5]:
from models.metrics import accuracy
print(accuracy(model, train_dl), accuracy(model, test_dl))

tensor(0.9312) tensor(0.9273)


In [6]:
model.to(device)
if use_sensitive_attr:
    all_X = dataset.data.to(device)
    all_X_conter = all_X.clone()
    all_X_conter[:, dataset.sensitive_idxs[0]] = 1 - all_X_conter[:, dataset.sensitive_idxs[0]]

    all_pred = model.get_prediction(all_X)
    all_pred_conter = model.get_prediction(all_X_conter)

    d_len = len(all_pred)
    n_unfair = (all_pred != all_pred_conter).sum().item()
    print(f'unfair ratio: {n_unfair/d_len} ({n_unfair}/{d_len})')

unfair ratio: 0.028863879603392754 (6122/212099)


In [7]:
data_gen.to('cpu')
distance_x_Causal = ProtectedSEDistances()
distance_x_LR = LogisticRegSensitiveSubspace()
distance_y = BinaryDistance()

if use_sensitive_attr:
    distance_x_Causal.fit(num_dims=dataset.dim_feature(), data_gen=data_gen, sensitive_idx=dataset.sensitive_idxs)
    chosen_dx = distance_x_Causal
else:
    sensitive_ = dataset.data[:, dataset.sensitive_idxs].to('cpu')
    distance_x_LR.fit(dataset.get_all_data().to('cpu'), data_gen=data_gen, data_SensitiveAttrs=sensitive_)
    chosen_dx = distance_x_LR

In [8]:
# x = torch.zeros(chosen_dx.num_dims)
# pert = 10*torch.diag(torch.ones_like(x))
# g = torch.zeros_like(x)
# for i in range(g.shape[0]):
#     g[i] = chosen_dx(x, x+pert[i])
# epsilon = (1/torch.min(g[g!=0])).item()
# print(g, epsilon)

In [9]:
epsilon = 1e8
unfair_metric = UnfairMetric(dx=chosen_dx, dy=distance_y, epsilon=epsilon)
model = model.to('cpu')

In [10]:
def show_result(result):
    pair, n_query = result[0], result[1]
    if len(result) == 3:
        print(f'n_iters = {result[2]}')
    print(n_query)
    # if pair != None:
    #     display(data_gen.feature_dataframe(data = pair), n_query)
    # else:
    #     display('not found')

In [11]:
# random.seed(422)
# torch.manual_seed(422)

# select_seeker = RandomSelectSeeker(model=model, unfair_metric=unfair_metric, data=dataset.get_all_data(), data_gen=data_gen)
# for _ in range(3):
#     show_result(select_seeker.seek(max_query=1e6))

In [12]:
# random.seed(422)
# torch.manual_seed(422)

# distribution_seeker = DistributionGenSeeker(model=model, unfair_metric=unfair_metric, data_gen=data_gen)
# for _ in range(3):
#     show_result(distribution_seeker.seek(max_query=1e6))

In [13]:
# random.seed(422)
# torch.manual_seed(422)

# range_seeker = RangeGenSeeker(model=model, unfair_metric=unfair_metric, data_gen=data_gen)
# for _ in range(3):
#     show_result(range_seeker.seek(max_query=1e5))

In [14]:
# random.seed(42)
# torch.manual_seed(42)

# test_seeker = WhiteboxSeeker(model=model, unfair_metric=unfair_metric, data_gen=data_gen)
# for i in range(3):
#     display(f'try: {i}')
#     show_result(test_seeker.seek(origin_lr=0.01, max_query=1e6, lamb=1))

In [15]:
random.seed(422)
torch.manual_seed(422)

test_seeker = BlackboxSeeker(model=model, unfair_metric=unfair_metric, data_gen=data_gen, g_range=1e-3, easy=False)
# show_result(white_seeker.seek())
for i in range(30):
    display(f'try: {i}')
    show_result(test_seeker.seek(origin_lr=0.1, max_query=1e5, lamb=1))

'try: 0'

100002


'try: 1'

100027


'try: 2'

...

100008


'try: 11'

100027


'try: 12'

KeyboardInterrupt: 

In [ ]:
random.seed(422)
torch.manual_seed(422)

test_seeker = BlackboxSeeker(model=model, unfair_metric=unfair_metric, data_gen=data_gen, easy=True)
# show_result(white_seeker.seek())
for i in range(3):
    display(f'try: {i}')
    show_result(test_seeker.seek(origin_lr=0.1, max_query=1e5, lamb=1))

'try: 0'

190


'try: 1'

185


'try: 2'

194


In [ ]:
# random.seed(422)
# torch.manual_seed(422)

# test_seeker = FoolSeeker(model=model, unfair_metric=unfair_metric, data_gen=data_gen, easy=True)

# for i in range(3):
#     display(f'try: {i}')
#     show_result(test_seeker.seek(origin_lr=1, max_query=1e6, lamb=1))